In [1]:
import cv2
import pyntcloud
import random
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import bundle_adjustment as b
import matching as m
import reconstruction as r

%matplotlib inline
mpl.rcParams['figure.dpi']= 200

In [2]:
### This cell does matching between images and outlier removal
n_imgs = 46 #46 if imgset = 'templering', 49 if imgset = 'Viking'

images, keypoints, descriptors, K = m.find_features(n_imgs, imgset='templering')

matcher = cv2.BFMatcher(cv2.NORM_L1)
matches = m.find_matches(matcher, keypoints, descriptors)
print('num_matches before outlier removal:', m.num_matches(matches))
m.print_num_img_pairs(matches)

matches = m.remove_outliers(matches, keypoints)
print("After outlier removal:")
m.print_num_img_pairs(matches)

img_adjacency, list_of_img_pairs = m.create_img_adjacency_matrix(n_imgs, matches)

num_matches before outlier removal: 59608
Number of img pairs is 1034 out of possible 1035
After outlier removal:
Number of img pairs is 243 out of possible 1035


In [6]:
### This cell initializes the reconstruction
best_pair = r.best_img_pair(img_adjacency, matches, keypoints, K, top_x_perc=0.2)
R0, t0, R1, t1, points3d_with_views = r.initialize_reconstruction(keypoints, matches, K, best_pair[0], best_pair[1])

R_mats = {best_pair[0]: R0, best_pair[1]: R1}
t_vecs = {best_pair[0]: t0, best_pair[1]: t1}

resected_imgs = [best_pair[0], best_pair[1]] 
unresected_imgs = [i for i in range(len(images)) if i not in resected_imgs] 
print('initial image pair:', resected_imgs)
avg_err = 0

Triangulating: 436 points.
initial image pair: [20, 23]


In [15]:
### This cell grows and refines the reconstruction 
BA_chkpts = [3,4,5,6] + [int(6*(1.34**i)) for i in range(25)]
while len(unresected_imgs) > 0:
    resected_idx, unresected_idx, prepend = r.next_img_pair_to_grow_reconstruction(n_imgs, best_pair, resected_imgs, unresected_imgs, img_adjacency)
    points3d_with_views, pts3d_for_pnp, pts2d_for_pnp, triangulation_status = r.get_correspondences_for_pnp(resected_idx, unresected_idx, points3d_with_views, matches, keypoints)
    if len(pts3d_for_pnp) < 12:
        print(f"{len(pts3d_for_pnp)} is too few correspondences for pnp. Skipping imgs resected:{resected_idx} and unresected:{unresected_idx}")
        print(f"Currently resected imgs: {resected_imgs}, unresected: {unresected_imgs}")
        continue

    R_res = R_mats[resected_idx]
    t_res = t_vecs[resected_idx]
    print(f"Unresected image: {unresected_idx}, resected: {resected_idx}")
    R_new, t_new = r.do_pnp(pts3d_for_pnp, pts2d_for_pnp, K)
    R_mats[unresected_idx] = R_new
    t_vecs[unresected_idx] = t_new
    if prepend == True: resected_imgs.insert(0, unresected_idx)
    else: resected_imgs.append(unresected_idx)
    unresected_imgs.remove(unresected_idx)
    pnp_errors, projpts, avg_err, perc_inliers = r.test_reproj_pnp_points(pts3d_for_pnp, pts2d_for_pnp, R_new, t_new, K)
    print(f"Average error of reprojecting points used to resect image {unresected_idx} back onto it is: {avg_err}")
    print(f"Fraction of Pnp inliers: {perc_inliers} num pts used in Pnp: {len(pnp_errors)}")
    
    if resected_idx < unresected_idx:
        kpts1, kpts2, kpts1_idxs, kpts2_idxs = r.get_aligned_kpts(resected_idx, unresected_idx, keypoints, matches, mask=triangulation_status)
        if np.sum(triangulation_status) > 0: #at least 1 point needs to be triangulated
            points3d_with_views, tri_errors, avg_tri_err_l, avg_tri_err_r = r.triangulate_points_and_reproject(R_res, t_res, R_new, t_new, K, points3d_with_views, resected_idx, unresected_idx, kpts1, kpts2, kpts1_idxs, kpts2_idxs, reproject=True)
    else:
        kpts1, kpts2, kpts1_idxs, kpts2_idxs = r.get_aligned_kpts(unresected_idx, resected_idx, keypoints, matches, mask=triangulation_status)
        if np.sum(triangulation_status) > 0: #at least 1 point needs to be triangulated
            points3d_with_views, tri_errors, avg_tri_err_l, avg_tri_err_r = r.triangulate_points_and_reproject(R_new, t_new, R_res, t_res, K, points3d_with_views, unresected_idx, resected_idx, kpts1, kpts2, kpts1_idxs, kpts2_idxs, reproject=True)
    
    if 0.8 < perc_inliers < 0.95 or 5 < avg_tri_err_l < 10 or 5 < avg_tri_err_r < 10: 
        #If % of inlers from Pnp is too low or triangulation error on either image is too high, bundle adjust
        points3d_with_views, R_mats, t_vecs = b.do_BA(points3d_with_views, R_mats, t_vecs, resected_imgs, keypoints, K, ftol=1e0)
        
    if len(resected_imgs) in BA_chkpts or len(unresected_imgs) == 0 or perc_inliers <= 0.8 or avg_tri_err_l >= 10 or avg_tri_err_r >= 10:
        #If % of inlers from Pnp is very low or triangulation error on either image is very high, bundle adjust with stricter tolerance
        points3d_with_views, R_mats, t_vecs = b.do_BA(points3d_with_views, R_mats, t_vecs, resected_imgs, keypoints, K, ftol=1e-1)
    
    av = 0
    for im in resected_imgs:
        p3d, p2d, avg_error, errors = r.get_reproj_errors(im, points3d_with_views, R_mats[im], t_vecs[im], K, keypoints, distCoeffs=np.array([]))
        print(f'Average reprojection error on image {im} is {avg_error} pixels')
        av += avg_error
    av = av/len(resected_imgs)
    print(f'Average reprojection error across all {len(resected_imgs)} resected images is {av} pixels')
    if len(resected_imgs) == 30: break

Unresected image: 25, resected: 24
rvec: [[ 0.50766114]
 [-2.10156888]
 [ 2.29867579]] 

tvec: [[ 0.0434132 ]
 [-1.80155935]
 [ 0.5155942 ]]
Average error of reprojecting points used to resect image 25 back onto it is: 0.7049232652085571
Fraction of Pnp inliers: 1.0 num pts used in Pnp: 388
Triangulating: 184 points.
Average reprojection error for just-triangulated points on image 24 is: 6.068759133733349 pixels.
Average reprojection error for just-triangulated points on image 25 is: 2.3039583504029206 pixels.
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.6929e+04                                    9.41e+05    
       1              3         8.6274e+03      8.30e+03       4.84e-01       2.64e+05    
`ftol` termination condition is satisfied.
Function evaluations 3, initial cost 1.6929e+04, final cost 8.6274e+03, first-order optimality 2.64e+05.
Average reprojection error on image 22 is 0.5204822007721859 pi

Unresected image: 16, resected: 17
rvec: [[0.02223558]
 [0.89711449]
 [2.92244099]] 

tvec: [[0.04332104]
 [1.52612557]
 [0.37933195]]
Average error of reprojecting points used to resect image 16 back onto it is: 0.7714300840380935
Fraction of Pnp inliers: 0.9905362776025236 num pts used in Pnp: 317
Triangulating: 174 points.
Average reprojection error for just-triangulated points on image 16 is: 0.8507336397063667 pixels.
Average reprojection error for just-triangulated points on image 17 is: 5.601516005877062 pixels.
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         2.0770e+04                                    1.15e+06    
       1              2         9.9349e+03      1.08e+04       1.72e+00       7.00e+05    
`ftol` termination condition is satisfied.
Function evaluations 2, initial cost 2.0770e+04, final cost 9.9349e+03, first-order optimality 7.00e+05.
Average reprojection error on image 22 is 0.4503236479

Unresected image: 30, resected: 29
rvec: [[-224.63737942]
 [ 246.59591333]
 [ -62.59257277]] 

tvec: [[ 0.26766877]
 [-2.96294274]
 [ 1.91192027]]
Average error of reprojecting points used to resect image 30 back onto it is: 0.5679263030803356
Fraction of Pnp inliers: 1.0 num pts used in Pnp: 307
Triangulating: 143 points.
Average reprojection error for just-triangulated points on image 29 is: 2.888277276969773 pixels.
Average reprojection error for just-triangulated points on image 30 is: 0.32108702199883465 pixels.
Average reprojection error on image 22 is 0.3838363647551715 pixels
Average reprojection error on image 21 is 0.19357493331989883 pixels
Average reprojection error on image 20 is 0.267425351326115 pixels
Average reprojection error on image 23 is 0.2351497947019424 pixels
Average reprojection error on image 24 is 0.16792269845799251 pixels
Average reprojection error on image 19 is 0.3241850056923351 pixels
Average reprojection error on image 25 is 0.1567349785024088 pixels


Average reprojection error on image 13 is 0.7684736601116378 pixels
Average reprojection error on image 31 is 0.825631201809194 pixels
Average reprojection error on image 12 is 0.6311110181212577 pixels
Average reprojection error on image 32 is 0.39060564374354056 pixels
Average reprojection error across all 21 resected images is 0.2846757841329606 pixels
Unresected image: 11, resected: 12
rvec: [[ 1.25347452]
 [-0.01066654]
 [-0.18293598]] 

tvec: [[0.32637766]
 [2.98615666]
 [2.14799688]]
Average error of reprojecting points used to resect image 11 back onto it is: 1.481981181654146
Fraction of Pnp inliers: 1.0 num pts used in Pnp: 71
Triangulating: 128 points.
Average reprojection error for just-triangulated points on image 11 is: 0.38045077461109367 pixels.
Average reprojection error for just-triangulated points on image 12 is: 0.36729486661168453 pixels.
Average reprojection error on image 22 is 0.38266310383661256 pixels
Average reprojection error on image 21 is 0.194621082674375

Average reprojection error on image 13 is 0.15735045376321005 pixels
Average reprojection error on image 31 is 0.1489785924633968 pixels
Average reprojection error on image 12 is 0.16049501009035227 pixels
Average reprojection error on image 32 is 0.14402683429523078 pixels
Average reprojection error on image 11 is 0.11297105517077824 pixels
Average reprojection error on image 33 is 0.12474588930162973 pixels
Average reprojection error on image 10 is 0.11658161590516758 pixels
Average reprojection error on image 34 is 0.29199178305972695 pixels
Average reprojection error across all 25 resected images is 0.18916903585263056 pixels
Unresected image: 9, resected: 10
rvec: [[0.16231767]
 [2.02355744]
 [2.21527702]] 

tvec: [[0.44964469]
 [3.14570113]
 [2.95370165]]
Average error of reprojecting points used to resect image 9 back onto it is: 0.36603166521965164
Fraction of Pnp inliers: 1.0 num pts used in Pnp: 133
Triangulating: 107 points.
Average reprojection error for just-triangulated p

Average reprojection error on image 30 is 0.16120404407319594 pixels
Average reprojection error on image 13 is 0.14099490882817806 pixels
Average reprojection error on image 31 is 0.14194798191983646 pixels
Average reprojection error on image 12 is 0.14627303099199573 pixels
Average reprojection error on image 32 is 0.12073153268654024 pixels
Average reprojection error on image 11 is 0.1227992299403102 pixels
Average reprojection error on image 33 is 0.11013150946542494 pixels
Average reprojection error on image 10 is 0.1396662934923137 pixels
Average reprojection error on image 34 is 0.09957671833746498 pixels
Average reprojection error on image 9 is 0.10314665958180447 pixels
Average reprojection error on image 35 is 0.1267859071245942 pixels
Average reprojection error on image 8 is 0.136220821482865 pixels
Average reprojection error across all 28 resected images is 0.17304491301495362 pixels
Unresected image: 36, resected: 35
rvec: [[-1.93865076]
 [ 0.01456945]
 [ 0.3077692 ]] 

tve

In [16]:
### This cell visualizes the pointcloud
num_voxels = 200 #Set to 100 for faster visualization, 200 for higher resolution.
x, y, z = [], [], []
for pt3 in points3d_with_views:
    if abs(pt3.point3d[0][0]) + abs(pt3.point3d[0][1]) + abs(pt3.point3d[0][2]) < 100:
        x.append(pt3.point3d[0][0])
        y.append(pt3.point3d[0][1])
        z.append(pt3.point3d[0][2])
vpoints = list(zip(x,y,z))
vpoints = np.array(vpoints)
vpoints_df = pd.DataFrame(data=vpoints, index=[f"{i}" for i in range(vpoints.shape[0])], columns=["x", "y","z"])
cloud = pyntcloud.PyntCloud(vpoints_df)
cloud.add_structure('voxelgrid', n_x=num_voxels,n_y=num_voxels,n_z=num_voxels)
cloud.structures[f'V([{num_voxels}, {num_voxels}, {num_voxels}],[None, None, None],True)'].plot(d=3)